In [1]:
import sys
sys.path.insert(0,'/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages' )
sys.path.insert(1,'/global/homes/b/bpb/metatlas/metatlas/compounds/not_used/')
import get_compounds_each_provider as import_compounds
import pandas as pd
# import sys
# sys.path.append('/global/project/projectdirs/openmsi/jupyterhub_libs/anaconda/lib/python2.7/site-packages')
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
import numpy as np


In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/ModelSEED/ModelSEEDDatabase/master/Biochemistry/compounds.master.tsv',sep='\t')


In [3]:
df.head()

,id,abbreviation,name,formula,mass,source,structure,charge,is_core,is_obsolete,linked_compound,is_cofactor,deltag,deltagerr,pka,pkb,abstract_compound,comprised_of,aliases
0,cpd00001,h2o,H2O,H2O,18,ModelSEED,InChI=1S/H2O/h1H2,0,1,0,null,0,-56.687,0.5,1:15.7,1:-1.8,null,null,null
1,cpd00002,atp,ATP,C10H13N5O13P3,504,ModelSEED,"InChI=1S/C10H16N5O13P3/c11-8-5-9(13-2-12-8)15(3-14-5)10-7(17)6(16)4(26-10)1-25-30(21,22)28-31(23,24)27-29(18,19)20/h2-4,6-7,10,16-17H,1H2,(H,21,22)(H,23,24)(H2,11,12,13)(H2,18,19,20)/t4-,6-,7-,10-/m1/s1",-3,1,0,null,0,-673.85,3.04314,26:0.84;22:2.95;14:13.03;29:1.83;30:7.72,15:3.97;14:-3.66;6:-3.48;9:-9.18,null,null,null
2,cpd00003,nad,NAD,C21H26N7O14P2,662,ModelSEED,"InChI=1S/C21H27N7O14P2/c22-17-12-19(25-7-24-17)28(8-26-12)21-16(32)14(30)11(41-21)6-39-44(36,37)42-43(34,35)38-5-10-13(29)15(31)20(40-10)27-3-1-2-9(4-27)18(23)33/h1-4,7-8,10-11,13-16,20-21,29-32H,5-6H2,(H5-,22,23,24,25,33,34,35,36,37)/t10-,11-,13-,14-,15-,16-,20-,21-/m1/s1",-1,1,0,null,0,-529.59,4.35693,17:1.8;18:2.56;6:12.32;25:11.56;35:13.12,6:-3.98;35:-3.66;25:-1.17;37:-3.48;41:3.94;43:-9.18,null,null,null
3,cpd00004,nadh,NADH,C21H27N7O14P2,663,ModelSEED,"InChI=1S/C21H29N7O14P2/c22-17-12-19(25-7-24-17)28(8-26-12)21-16(32)14(30)11(41-21)6-39-44(36,37)42-43(34,35)38-5-10-13(29)15(31)20(40-10)27-3-1-2-9(4-27)18(23)33/h1,3-4,7-8,10-11,13-16,20-21,29-32H,2,5-6H2,(H2,23,33)(H,34,35)(H,36,37)(H2,22,24,25)/t10-,11-,13-,14-,15-,16-,20-,21-/m1/s1",-2,1,0,null,0,-524.32,4.26795,18:14.3;14:13.31;22:1.8;26:2.56;36:12.69,36:-3.6;18:-3.51;14:-3.66;9:-3.48;15:3.94;6:-9.18,null,null,null
4,cpd00005,nadph,NADPH,C21H26N7O17P3,742,ModelSEED,"InChI=1S/C21H30N7O17P3/c22-17-12-19(25-7-24-17)28(8-26-12)21-16(44-46(33,34)35)14(30)11(43-21)6-41-48(38,39)45-47(36,37)40-5-10-13(29)15(31)20(42-10)27-3-1-2-9(4-27)18(23)32/h1,3-4,7-8,10-11,13-16,20-21,29-31H,2,5-6H2,(H2,23,32)(H,36,37)(H,38,39)(H2,22,24,25)(H2,33,34,35)/t10-,11-,13-,14-,15-,16-,20-,21-/m1/s1",-4,1,0,null,0,-736.82,4.25788,19:6.71;30:2.61;40:13.16;18:2;26:1.45,40:-3.6;12:-3.48;22:3.94;11:-9.18,null,null,null


In [4]:
df.shape

(27693, 19)

In [5]:
def make_mol_from_smiles_and_inchi(row):
    mol= []
    if row['structure']:
        mol = Chem.MolFromInchi(row['structure'].encode('utf-8'))
    if mol:
        return mol

df.ROMol = df.apply(make_mol_from_smiles_and_inchi, axis=1)

def desalt_compounds_in_dataframe(x):
    if x:
        if x.GetNumAtoms()>1:
            c = import_compounds.desalt(x)
            if c[1]:
                return c[0]
            else:
                return x
    
df.ROMol = df.ROMol.apply(desalt_compounds_in_dataframe)


import_compounds = reload(import_compounds)
def neutralize_compounds_in_dataframe(x):
    if x:
        if x.GetNumAtoms()> 0:
            neutral_mol = []
            try:
                c = import_compounds.NeutraliseCharges(x)
                neutral_mol = c[0]
            except:
                pass
            if neutral_mol:
                return neutral_mol
            
df.ROMol = df.ROMol.apply(neutralize_compounds_in_dataframe)

def calculate_num_radicals_in_dataframe(x):
    num_radicals = 0.0
    if x:
        num_radicals = Descriptors.NumRadicalElectrons(x)        
    return num_radicals

def calculate_formula_in_dataframe(x):
    formula = ''
    if x:
        formula = rdMolDescriptors.CalcMolFormula(x)
    return formula

def calculate_monoisotopic_mw_in_dataframe(x):
    mw = 0.0
    if x:
        mw = Descriptors.ExactMolWt(x)
    return mw

def calculate_inchi_in_dataframe(x):
    inchi = ''
    if x:
        try:
            inchi = Chem.MolToInchi(x)
        except:
            pass#This fails when can't kekulize mol
    return inchi

def calculate_flattened_inchi_in_dataframe(x):
    flattened_inchi = ''
    if x:
        sm = Chem.MolToSmiles(x).replace('@','')
        flattened_rdkit_mol = Chem.MolFromSmiles(sm)
        try:
            flattened_inchi = Chem.MolToInchi(flattened_rdkit_mol)
        except:
            pass#This fails when can't kekulize mol
    return flattened_inchi

def calculate_inchikey_in_dataframe(x):
    ik = ''
    if x:
        try:
            ik = Chem.InchiToInchiKey(x)
        except:
            pass#This fails when can't kekulize mol.  Carbo-cations are the culprit usually. 
    return ik

def calculate_charge_in_dataframe(x):
    if x:
        my_charge = Chem.GetFormalCharge(x)
        return my_charge
            
df['charge'] = df.ROMol.apply(calculate_charge_in_dataframe)
df['formula'] = df.ROMol.apply(calculate_formula_in_dataframe)
df['monoisotopoic_mw'] = df.ROMol.apply(calculate_monoisotopic_mw_in_dataframe)
df['num_radicals'] = df.ROMol.apply(calculate_num_radicals_in_dataframe)

df['metatlas_inchi'] = df.ROMol.apply(calculate_inchi_in_dataframe)
df['metatlas_inchi_key'] = df.metatlas_inchi.apply(calculate_inchikey_in_dataframe)

df['flat_inchi'] = df.ROMol.apply(calculate_flattened_inchi_in_dataframe)
df['flat_inchikey'] = df.flat_inchi.apply(calculate_inchikey_in_dataframe)


In [11]:
df2 = pd.read_pickle('https://github.com/biorack/metatlas/blob/master/metatlas/compounds/kbase_compounds_with_neutral_compound_information.pkl')

IOError: [Errno 2] No such file or directory: 'https://github.com/biorack/metatlas/blob/master/metatlas/compounds/kbase_compounds_with_neutral_compound_information.pkl'

In [ ]:
# df = df.where((pd.notnull(metatlas_df)), None)


In [ ]:
# df = df[metatlas_df.metatlas_inchi_key != '']


In [15]:
df.to_csv('kbase_compounds_with_neutral_compound_information.tab',sep='\t',index=False)
df.head()

,id,abbreviation,name,formula,mass,source,structure,charge,is_core,is_obsolete,...,pkb,abstract_compound,comprised_of,aliases,monoisotopoic_mw,num_radicals,metatlas_inchi,metatlas_inchi_key,flat_inchi,flat_inchikey
0,cpd00001,h2o,H2O,,18,ModelSEED,InChI=1S/H2O/h1H2,NaN,1,0,...,1:-1.8,null,null,null,0.000000,0.0,,,,
1,cpd00002,atp,ATP,C10H16N5O13P3,504,ModelSEED,"InChI=1S/C10H16N5O13P3/c11-8-5-9(13-2-12-8)15(3-14-5)10-7(17)6(16)4(26-10)1-25-30(21,22)28-31(23,24)27-29(18,19)20/h2-4,6-7,10,16-17H,1H2,(H,21,22)(H,23,24)(H2,11,12,13)(H2,18,19,20)/t4-,6-,7-,10-/m1/s1",0.0,1,0,...,15:3.97;14:-3.66;6:-3.48;9:-9.18,null,null,null,506.995745,0.0,"InChI=1S/C10H16N5O13P3/c11-8-5-9(13-2-12-8)15(3-14-5)10-7(17)6(16)4(26-10)1-25-30(21,22)28-31(23,24)27-29(18,19)20/h2-4,6-7,10,16-17H,1H2,(H,21,22)(H,23,24)(H2,11,12,13)(H2,18,19,20)/t4-,6-,7-,10-/m1/s1",ZKHQWZAMYRWXGA-KQYNXXCUSA-N,"InChI=1S/C10H16N5O13P3/c11-8-5-9(13-2-12-8)15(3-14-5)10-7(17)6(16)4(26-10)1-25-30(21,22)28-31(23,24)27-29(18,19)20/h2-4,6-7,10,16-17H,1H2,(H,21,22)(H,23,24)(H2,11,12,13)(H2,18,19,20)",ZKHQWZAMYRWXGA-UHFFFAOYSA-N
2,cpd00003,nad,NAD,C21H28N7O14P2+,662,ModelSEED,"InChI=1S/C21H27N7O14P2/c22-17-12-19(25-7-24-17)28(8-26-12)21-16(32)14(30)11(41-21)6-39-44(36,37)42-43(34,35)38-5-10-13(29)15(31)20(40-10)27-3-1-2-9(4-27)18(23)33/h1-4,7-8,10-11,13-16,20-21,29-32H,5-6H2,(H5-,22,23,24,25,33,34,35,36,37)/t10-,11-,13-,14-,15-,16-,20-,21-/m1/s1",1.0,1,0,...,6:-3.98;35:-3.66;25:-1.17;37:-3.48;41:3.94;43:-9.18,null,null,null,664.116398,0.0,"InChI=1S/C21H27N7O14P2/c22-17-12-19(25-7-24-17)28(8-26-12)21-16(32)14(30)11(41-21)6-39-44(36,37)42-43(34,35)38-5-10-13(29)15(31)20(40-10)27-3-1-2-9(4-27)18(23)33/h1-4,7-8,10-11,13-16,20-21,29-32H,5-6H2,(H5-,22,23,24,25,33,34,35,36,37)/p+1/t10-,11-,13-,14-,15-,16-,20-,21-/m1/s1",BAWFJGJZGIEFAR-NNYOXOHSSA-O,"InChI=1S/C21H27N7O14P2/c22-17-12-19(25-7-24-17)28(8-26-12)21-16(32)14(30)11(41-21)6-39-44(36,37)42-43(34,35)38-5-10-13(29)15(31)20(40-10)27-3-1-2-9(4-27)18(23)33/h1-4,7-8,10-11,13-16,20-21,29-32H,5-6H2,(H5-,22,23,24,25,33,34,35,36,37)/p+1",BAWFJGJZGIEFAR-UHFFFAOYSA-O
3,cpd00004,nadh,NADH,C21H29N7O14P2,663,ModelSEED,"InChI=1S/C21H29N7O14P2/c22-17-12-19(25-7-24-17)28(8-26-12)21-16(32)14(30)11(41-21)6-39-44(36,37)42-43(34,35)38-5-10-13(29)15(31)20(40-10)27-3-1-2-9(4-27)18(23)33/h1,3-4,7-8,10-11,13-16,20-21,29-32H,2,5-6H2,(H2,23,33)(H,34,35)(H,36,37)(H2,22,24,25)/t10-,11-,13-,14-,15-,16-,20-,21-/m1/s1",0.0,1,0,...,36:-3.6;18:-3.51;14:-3.66;9:-3.48;15:3.94;6:-9.18,null,null,null,665.124772,0.0,"InChI=1S/C21H29N7O14P2/c22-17-12-19(25-7-24-17)28(8-26-12)21-16(32)14(30)11(41-21)6-39-44(36,37)42-43(34,35)38-5-10-13(29)15(31)20(40-10)27-3-1-2-9(4-27)18(23)33/h1,3-4,7-8,10-11,13-16,20-21,29-32H,2,5-6H2,(H2,23,33)(H,34,35)(H,36,37)(H2,22,24,25)/t10-,11-,13-,14-,15-,16-,20-,21-/m1/s1",BOPGDPNILDQYTO-NNYOXOHSSA-N,"InChI=1S/C21H29N7O14P2/c22-17-12-19(25-7-24-17)28(8-26-12)21-16(32)14(30)11(41-21)6-39-44(36,37)42-43(34,35)38-5-10-13(29)15(31)20(40-10)27-3-1-2-9(4-27)18(23)33/h1,3-4,7-8,10-11,13-16,20-21,29-32H,2,5-6H2,(H2,23,33)(H,34,35)(H,36,37)(H2,22,24,25)",BOPGDPNILDQYTO-UHFFFAOYSA-N
4,cpd00005,nadph,NADPH,C21H30N7O17P3,742,ModelSEED,"InChI=1S/C21H30N7O17P3/c22-17-12-19(25-7-24-17)28(8-26-12)21-16(44-46(33,34)35)14(30)11(43-21)6-41-48(38,39)45-47(36,37)40-5-10-13(29)15(31)20(42-10)27-3-1-2-9(4-27)18(23)32/h1,3-4,7-8,10-11,13-16,20-21,29-31H,2,5-6H2,(H2,23,32)(H,36,37)(H,38,39)(H2,22,24,25)(H2,33,34,35)/t10-,11-,13-,14-,15-,16-,20-,21-/m1/s1",0.0,1,0,...,40:-3.6;12:-3.48;22:3.94;11:-9.18,null,null,null,745.091102,0.0,"InChI=1S/C21H30N7O17P3/c22-17-12-19(25-7-24-17)28(8-26-12)21-16(44-46(33,34)35)14(30)11(43-21)6-41-48(38,39)45-47(36,37)40-5-10-13(29)15(31)20(42-10)27-3-1-2-9(4-27)18(23)32/h1,3-4,7-8,10-11,13-16,20-21,29-31H,2,5-6H2,(H2,23,32)(H,36,37)(H,38,39)(H2,22,24,25)(H2,33,34,35)/t10-,11-,13-,14-,15-,16-,20-,21-/m1/s1",ACFIXJIJDZMPPO-NNYOXOHSSA-N,"InChI=1S/C21H30N7O17P3/c22-17-12-19(25-7-24-17)28(8-26-12)21-16(44-46(33,34)35)14(30)11(43-21)6-41-48(38,3

In [ ]:
# idx = metatlas_df.metatlas_inchi_key.isin(df.metatlas_inchi_key)

In [ ]:
# not_found = metatlas_df[~idx]

In [ ]:
# not_found = not_found.drop_duplicates()

In [ ]:
# not_found.head(20)

In [ ]:
# not_found.to_csv('metatlas_compounds_not_found.csv')
# not_found.to_pickle('metatlas_compounds_not_found.pkl')

In [ ]:
# not_found.shape